In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import os

api = KaggleApi()
api.authenticate()

competition_name = 'home-credit-credit-risk-model-stability'
api.competition_download_files(competition_name, path='./Data/')

zip_file_path = f'./Data/{competition_name}.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('./Data/')

os.remove(zip_file_path)

In [2]:
import polars as pl

train_base = pl.read_csv("Data/csv_files/train/train_base.csv")

train_static_0_0 = pl.read_csv("Data/csv_files/train/train_static_0_0.csv")
train_static_0_1 = pl.read_csv("Data/csv_files/train/train_static_0_1.csv")

In [3]:
columns_to_float = [
    'amtinstpaidbefduel24m_4187115A',
    'avgdbddpdlast3m_4187120P',
    'avgdbdtollast24m_4525197P',
    'avglnamtstart24m_4525187A',
    'avgoutstandbalancel6m_4187114A',
    'avgpmtlast12m_4525200A',
    'maxlnamtstart6m_4525199A',
    'maxoutstandbalancel12m_4187113A',
    'maxpmtlast3m_4525190A',
    'mindbdtollast24m_4525191P',
    'numinstlswithdpd5_4187116L',
    'numinstmatpaidtearly2d_4499204L',
    'numinstpaid_4499208L',
    'numinstpaidearly3dest_4493216L',
    'numinstpaidearly5dest_4493211L',
    'numinstpaidearly5dobd_4499205L',
    'numinstpaidearlyest_4493214L',
    'numinstpaidlastcontr_4325080L',
    'numinstregularpaidest_4493210L',
    'numinsttopaygrest_4493213L',
    'numinstunpaidmaxest_4493212L',
    'sumoutstandtotalest_4493215A',
    'totinstallast1m_4525188A'
]

special_casts = {
    'clientscnt_136L': pl.Float64,
    'maxdbddpdtollast6m_4187119P': pl.Float64
}

def convert_columns_to_float(df, columns):
    for column in columns:
        if column in df.columns:
            df = df.with_columns(pl.col(column).cast(pl.Float64).alias(column))
    return df

def convert_columns(df, columns_to_float, special_casts):
    for column in columns_to_float:
        if column in df.columns:
            df = df.with_columns(pl.col(column).cast(pl.Float64).alias(column))
    for column, dtype in special_casts.items():
        if column in df.columns:
            df = df.with_columns(pl.col(column).cast(dtype).alias(column))
    return df

train_static_0_0_converted = convert_columns(train_static_0_0, columns_to_float, special_casts)
train_static_0_1_converted = convert_columns(train_static_0_1, columns_to_float, special_casts)

In [4]:
train_static_0_1_converted = train_static_0_1_converted.with_columns(
    pl.when(pl.col("isbidproductrequest_292L") == "true")
    .then(True)
    .when(pl.col("isbidproductrequest_292L") == "false")
    .then(False)
    .otherwise(pl.col("isbidproductrequest_292L").is_null())
    .alias("isbidproductrequest_292L")
)

train_static_0_1_converted = train_static_0_1_converted.with_columns(
    pl.col("isbidproductrequest_292L").cast(pl.Boolean).alias("isbidproductrequest_292L")
)

In [5]:
train_static_0 = pl.concat([train_static_0_0_converted, train_static_0_1_converted])

In [6]:
train_static_cb_0 = pl.read_csv("Data/csv_files/train/train_static_cb_0.csv")

In [7]:
train_static = train_static_0.join(
    train_static_cb_0,
    on="case_id",
    how="left"
)

In [8]:
train_DEPTH_0 = train_base.join(
    train_static,
    on="case_id",
    how="outer"
)

In [9]:
train_DEPTH_0.shape

(1526659, 225)

In [11]:
directory = r"C:\Users\afise\.git\CreditRiskModel\Merged_Data"
filename = "train_DEPTH_0.csv"
file_path = os.path.join(directory, filename)

if not os.path.exists(directory):
    os.makedirs(directory)

train_DEPTH_0.write_csv(file_path)